In [1]:
import numpy as np
import pulp 

In [2]:
users_max_id = 6040
movie_max_id = 3952
correlations_data = np.fromfile('./correlations_data_set_saved', dtype=float)
correlations_data=correlations_data.reshape(movie_max_id+1,movie_max_id+1)

correlations_indicator = np.load('./correlations_indicator_set_saved.npy')
correlations_indicator=correlations_indicator.reshape(movie_max_id+1,movie_max_id+1)

bad_movies = np.load('./subsets/bad_movies.npy')
bad_movies = bad_movies.reshape((-1)).tolist()

subset_size=100
Indecies = np.array([x for x in range(0,movie_max_id+1)])

In [9]:
permutation= np.loadtxt('./subsets/randomsubset11')
permutation=permutation.reshape((-1)).astype(int)
#permutation = np.array(range(0,101))
bad_indices = np.zeros((permutation.shape[0]), dtype=bool)
for movie in bad_movies: 
    bad_indices = np.logical_or(permutation==movie,bad_indices)
permutation=permutation[~(bad_indices.reshape(-1))]
subset_size = permutation.shape[0]
ajdacency_matrix=np.zeros((subset_size+1,subset_size+1))
ajdacency_matrix=(correlations_indicator[permutation,:])[:,permutation]
probabilty_matrix= np.zeros((subset_size+1,subset_size+1))
probabilty_matrix = (correlations_data[permutation,:])[:,permutation]
print(permutation)
print(subset_size-1)

[   0 3572 1383 2899 1151 1854 3046 2758  422 3135 3250 3738 3061 3454
 2032  211   52  257   84 1007 1880 2019  944 2764 3536 1603 2940 3751
 2755  306 3820  116  240 2834 1199 1766 3213 3926  365 3880 3943 1358
   10 1711  519 3571 2502 1866 2116 1958 1747 1268 1546  181 2658  102
 1493 1150  573 3867 3093  743  352  923 2161 2171  941  610 2149 3499
 1813 2072 3017 3073 1779 1539 2598 2005 3940  236 3946 1061  467  461
 1332  524 3610]
86


In [10]:
temp_ajd = ajdacency_matrix[1:,1:]
temp_prob = np.log(1/(probabilty_matrix[1:,1:]+1e-10))
lmbda=10

In [11]:
ILP = pulp.LpProblem("find clusters",pulp.LpMinimize)
x={}
xi={}
for i in range(0,subset_size-1):
    for j in range(i+1,subset_size-1):
        x[str(i)+','+str(j)]=pulp.LpVariable('x'+str(i)+str(j), cat='Continuous',lowBound=0.0,upBound=1.0)
        xi[str(i)+','+str(j)]=pulp.LpVariable('xi'+str(i)+str(j), cat='Continuous',lowBound=0.0,upBound=1.0)

ILP+=pulp.LpAffineExpression([(x[str(i)+','+str(j)],temp_ajd[i,j]*temp_prob[i,j])
                             for i in range(0,subset_size-1) for j in range(i+1,subset_size-1)])
ILP+=pulp.LpAffineExpression([(xi[str(i)+','+str(j)] ,lmbda*(1-temp_ajd[i,j])*(temp_prob[i,j]))
                             for i in range(0,subset_size-1) for j in range(i+1,subset_size-1)])

for i in range(0,subset_size-1):
    for j in range(i+1,subset_size-1):
        for w in range(0,subset_size-1):
            if w!=j and w!=i:
                ILP+=x[str(i)+','+str(j)] <= x[str(min(i,w))+','+str(max(i,w))]+ x[str(min(j,w))+','+str(max(w,j))]
        ILP+=xi[str(i)+','+str(j)] == 1-x[str(i)+','+str(j)]

C:\ProgramData\Anaconda3\lib\site-packages\pulp\pulp.py:1349: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [12]:
ILP.solve()

1

In [1]:
# for i in range(0,subset_size-1):
#     for j in range(i+1,subset_size-1):
#         print(1-pulp.value(x[str(i)+','+str(j)]),temp_ajd[i,j])

In [398]:
clustering_vector = np.zeros((10,movie_max_id+1))
for i in range(0,10):
    current_cluster_number = 0
    W = np.zeros(subset_size-1)
    ajdacency_matrix_copy = np.copy(ajdacency_matrix)
    while(np.sum(1-W)!=0):
        v = np.random.choice(np.argwhere(W==0).reshape(-1),size=1)[0]
        current_cluster_number=current_cluster_number+1
        W[v]=1
        clustering_vector[i,ajdacency_matrix_copy[0,v+1].astype(int)]=current_cluster_number                 
        for j in range(1,subset_size-1):
            if v!=j:
                rand= np.random.rand(1)
                if rand<=1-pulp.value(x[str(min(v,j))+','+str(max(j,v))]):
                    clustering_vector[i,ajdacency_matrix_copy[0,j+1].astype(int)]=current_cluster_number  
                    W[j]=1

In [399]:
# data_clusters =np.unique(clustering_vector)
# sizes=[]
# for x in data_clusters[1:]:
#     cluster_size = np.sum((clustering_vector==x).astype(int))
#     sizes.append((x,cluster_size))
# print(sizes)
# print(subset_size-1)

In [400]:
costs=[]
for i in range(0,10):
    cost=0
    data_clusters = np.unique(clustering_vector[i,:])
    for c in range(1,len(data_clusters)+1):
        cluster = (clustering_vector[i,:] == c)
        if np.sum(cluster.astype(int))==1:
            cost=cost + np.log(1/(correlations_data[Indecies[cluster][0],Indecies[cluster][0]]))
        else:
            c_i=0
            for t in Indecies[cluster]:
                for j in Indecies[cluster]:
                    if t!=j:
                        if correlations_data[t,j]==0:
                                correlations_data[t,j]=correlations_data[j,t]
                        c_i=c_i+np.log(1/correlations_data[t,j])
            cost=cost+(c_i/2)*(1/(np.sum(cluster)-1))
    costs.append(cost)
np_costs = np.array(costs)
print('clustering {} is the best, with cost: {}'.format(np.argmin(np_costs),np.min(np_costs)))

clustering 8 is the best, with cost: 460.76719287323425
